In [1]:
import os

In [2]:
from pyiron_base import Settings, InputList, Executable

In [3]:
from pyiron import Project, pyiron_to_ase, ase_to_pyiron

In [4]:
from pyiron.atomistics.job.atomistic import AtomisticGenericJob

In [5]:
from pycp2k import CP2K
from ase.lattice.cubic import Diamond

In [6]:
s = Settings()

In [7]:
pyiron_dict = {
    "global": {
        "run_type": "ENERGY_FORCE"
    },
    "force_eval": {
        "method": "Quickstep", 
        "print_forces_section": "ON"
    },
    "dft": {
        "qs": {
            "eps": 1.0E-10
        },
        "mgrid": {
            "ngrids": 4,
            "cutoff": 300,
            "rel_cutoff": 60
        },
        "xc": {
            "functional": "PADE"
        }
    },
    "scf": {
        "scf_guess": "ATOMIC",
        "eps": 1.0E-7,
        "max": 300,
        "diagonalization": {
            "algorthim": "STANDARD"
        },
        "mixing": {
            "method": "BROYDEN_MIXING",
            "alpha": 0.4, 
            "n_broyden": 8,
        }
    },
    "kind": {
        "basis_set": "DZVP-GTH-PADE",
        "potential": "GTH-PADE-q4"
    }
}

In [8]:
class Cp2k(AtomisticGenericJob): 
    def __init__(self, project, job_name):
        super(Cp2k, self).__init__(project, job_name)
        self.__name__ = "Cp2k"
        self._executable = Executable(
            codename=self.__name__, 
            path_binary_codes=s.resource_paths, 
            module="cp2k"
        )
        self.input = InputList(pyiron_dict)
        
    def to_hdf(self, hdf=None, group_name=None):
        """
        Store the ExampleJob object in the HDF5 File

        Args:
            hdf (ProjectHDFio): HDF5 group object - optional
            group_name (str): HDF5 subgroup name - optional
        """
        super(Cp2k, self).to_hdf(hdf=hdf, group_name=group_name)
        self._structure_to_hdf()
        with self.project_hdf5.open("input") as hdf5_input:
            self.input.to_hdf(hdf5_input)

    def from_hdf(self, hdf=None, group_name=None):
        """
        Restore the ExampleJob object in the HDF5 File

        Args:
            hdf (ProjectHDFio): HDF5 group object - optional
            group_name (str): HDF5 subgroup name - optional
        """
        super(Cp2k, self).from_hdf(hdf=hdf, group_name=group_name)
        self._structure_from_hdf()
        with self.project_hdf5.open("input") as hdf5_input:
            self.input.from_hdf(hdf5_input)
            
    def write_input(self):
        calc = CP2K()
        calc.working_directory = self.working_directory
        calc.project_name = "pyiron"
        CP2K_INPUT = calc.CP2K_INPUT
        GLOBAL = CP2K_INPUT.GLOBAL
        FORCE_EVAL = CP2K_INPUT.FORCE_EVAL_add()  # Repeatable items have to be first created
        SUBSYS = FORCE_EVAL.SUBSYS
        DFT = FORCE_EVAL.DFT
        SCF = DFT.SCF
        GLOBAL.Run_type = self.input["global"]["run_type"]
        FORCE_EVAL.Method = self.input["force_eval"]["method"]
        FORCE_EVAL.PRINT.FORCES.Section_parameters = self.input["force_eval"]["print_forces_section"]
        DFT.Basis_set_file_name = os.path.join(s.resource_paths[0], "cp2k", "potentials", "BASIS_SET")
        DFT.Potential_file_name = os.path.join(s.resource_paths[0], "cp2k", "potentials", "GTH_POTENTIALS")
        DFT.QS.Eps_default = self.input["dft"]["qs"]["eps"]
        DFT.MGRID.Ngrids = self.input["dft"]["mgrid"]["ngrids"]
        DFT.MGRID.Cutoff = self.input["dft"]["mgrid"]["cutoff"]
        DFT.MGRID.Rel_cutoff = self.input["dft"]["mgrid"]["rel_cutoff"]
        DFT.XC.XC_FUNCTIONAL.Section_parameters = self.input["dft"]["xc"]["functional"]
        SCF.Scf_guess = self.input["scf"]["scf_guess"]
        SCF.Eps_scf = self.input["scf"]["eps"]
        SCF.Max_scf = self.input["scf"]["max"]
        SCF.DIAGONALIZATION.Section_parameters = "ON"
        SCF.DIAGONALIZATION.Algorithm = self.input["scf"]["diagonalization"]["algorthim"]
        SCF.MIXING.Section_parameters = "T"
        SCF.MIXING.Method = self.input["scf"]["mixing"]["method"]
        SCF.MIXING.Alpha = self.input["scf"]["mixing"]["alpha"]
        SCF.MIXING.Nbroyden = self.input["scf"]["mixing"]["n_broyden"]
        for el in set(self.structure.get_chemical_symbols()):
            KIND = SUBSYS.KIND_add(el)
        KIND.Basis_set = self.input["kind"]["basis_set"]
        KIND.Potential = self.input["kind"]["potential"]
        calc.create_cell(SUBSYS, pyiron_to_ase(self.structure))
        calc.create_coord(SUBSYS, pyiron_to_ase(self.structure))
        calc.write_input_file()
        
    def collect_output(self):
        pass

In [9]:
pr = Project("test")

In [10]:
pr.remove_jobs_silently(recursive=True)

In [11]:
job = pr.create_job(Cp2k, "cp2k_demo")
job.structure = ase_to_pyiron(Diamond(
    directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
    symbol='Si',
    latticeconstant=5.430697500,
    size=(1, 1, 1)
))
job.run()

The job cp2k_demo was saved and received the ID: 1


In [12]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,1,finished,Si8,cp2k_demo,/cp2k_demo,/home/jovyan/,test/,2020-10-25 11:36:08.110706,2020-10-25 11:37:35.853247,87.0,pyiron@jupyter-jan-2djanssen-2dpyiron-2dcp2k-2dkffzhg5p#1,Cp2k,0.1,None,None


In [13]:
job.decompress()

In [14]:
job["pyiron.out"]

[' DBCSR| CPU Multiplication driver                                           BLAS\n',
 ' DBCSR| Multrec recursion limit                                              512\n',
 ' DBCSR| Multiplication stack size                                           1000\n',
 ' DBCSR| Maximum elements for images                                    UNLIMITED\n',
 ' DBCSR| Multiplicative factor virtual images                                   1\n',
 ' DBCSR| Use multiplication densification                                       T\n',
 ' DBCSR| Multiplication size stacks                                             3\n',
 '\n',
 '\n',
 '  **** **** ******  **  PROGRAM STARTED AT               2020-10-25 11:36:08.327\n',
 ' ***** ** ***  *** **   PROGRAM STARTED ON        jupyter-jan-2djanssen-2dpyiron\n',
 ' **    ****   ******    PROGRAM STARTED BY                                jovyan\n',
 ' ***** **    ** ** **   PROGRAM PROCESS ID                                  4929\n',
 '  **** **  *******  **  PRO